In [ ]:
import robosuite as suite
from robosuite.wrappers.gym_wrapper import GymWrapper
import gymnasium as gym
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor
import time
from stable_baselines3.common.env_util import make_vec_env


In [ ]:
import os
filename = 'tmp/gym/cnninput'
cwd = os.getcwd()
new_folder = os.path.join(cwd, filename)
os.makedirs(new_folder, exist_ok=True)

In [ ]:
# create environment instance
env = GymWrapper(suite.make(
    env_name="Lift", # try with other tasks like "Stack" and "Door"
    robots="UR5e",  # try with other robots like "Sawyer" and "Jaco"
    has_renderer=True,
    has_offscreen_renderer=True,
    use_object_obs=True,                   # don't provide object observations to agent
    use_camera_obs=True,
    camera_names="robot0_eye_in_hand",      # use "agentview" camera for observations
    camera_heights=84,                      # image height
    camera_widths=84,                       # image width
    reward_shaping=True,                    # use a dense reward signal for learning
    horizon = 500,
    control_freq=20,                        # control should happen fast enough so that simulation looks smooth
), ['robot0_eye_in_hand_image'])
env = Monitor(env, filename)
# ), ['robot0_eye_in_hand_image'])

In [ ]:
env.observation_space

In [ ]:
from stable_baselines3 import PPO
model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=2048, progress_bar=True, log_interval=5)

In [ ]:
model.save(filename)

In [ ]:
obs = env.reset()[0]
action_list = []
for i in range(500):
    action, state = model.predict(obs)
    action_list.append(action)
    obs, reward, done,done, info = env.step(action)
    env.render()
    time.sleep(0.2)
    if done:
        break
env.close()

In [ ]:
from stable_baselines3 import PPO
model = PPO.load('trained_model', env=env)
model.learn(total_timesteps=100000, progress_bar=True, log_interval=10)
model.save('trained_model')


In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import robosuite.macros as macros
macros.IMAGE_CONVENTION = "opencv"
env = GymWrapper(suite.make(
    env_name="Lift", # try with other tasks like "Stack" and "Door"
    robots="UR5e",  # try with other robots like "Sawyer" and "Jaco"
    has_renderer=False,
    ignore_done = True,
    has_offscreen_renderer=True,
    use_object_obs=True,                   # don't provide object observations to agent
    use_camera_obs=True,
    camera_names="agentview",      # use "agentview" camera for observations
    camera_heights=512,                      # image height
    camera_widths=512,                       # image width
    reward_shaping=True,                    # use a dense reward signal for learning
    horizon = 500,
    control_freq=20,                        # control should happen fast enough so that simulation looks smooth
))

In [ ]:
import imageio

writer = imageio.get_writer(f'{filename}/video.mp4', fps=20)
obs = env.reset()[0]
for i,v in enumerate(action_list):
    obs, reward, done,done, info = env.step(v)
    frame = obs["agentview_image"]
    writer.append_data(frame)
    print("Saving frame #{}".format(i))
    if done:
        break
writer.close()
env.close()